In [1]:
DL = '/Users/damiansp/Downloads'

In [2]:
!rm -rf $DL/*

In [3]:
from datetime import datetime, timedelta
import os

import numpy as np
import pandas as pd
import xlrd

In [4]:
SYMBOLS = [
    'AAPL', 'ABMD', 'ABNB', 'ADBE', 'ADYEY', 'AMD', 'AMGN', 'AMZN',
    'ANET', 'ASML', 'AXON', 'BROS', 'BYND', 'CHWY', 'CRWD', 'DDOG',
    'DOCS', 'DOCU', 'DUOL', 'EDIT', 'ETSY', 'FLGT', 'GH', 'GMED',
    'GOOG', 'IDXX', 'INTG', 'ISRG', 'JD', 'LRCX', 'LSPD', 'LULU', 'MDB',
    'MELI', 'MNST', 'MSFT', 'MTCH', 'NVCR', 'NVDA', 'ODFL', 'OKTA',
    'OPEN', 'PAC', 'PANW', 'PAYC', 'PINS', 'PTON', 'PYPL', 'QDEL',
    'RBLX', 'RDFN', 'RGEN', 'RIVN', 'ROKU', 'RVLV', 'SE', 'SHOP',
    'SNOW', 'TASK', 'TDOC', 'TEAM', 'TREX', 'TSLA', 'TTD', 'TTWO',
    'TWLO', 'TWTR', 'TYL', 'UPST', 'VEEV', 'VLD', 'VRTX', 'XPEV', 'ZEN',
    'ZM']
LINGERERS = [
    'AMEH', 'AMRK', 'APT', 'CTSH', 'CUBE', 'DTST', 'EP', 'EXR', 'FCPT',
    'FCUV', 'GBOX', 'GGE', 'GME', 'INTZ', 'KREF', 'LFMD', 'MCVT',
    'MNMD', 'PETV', 'POWW', 'PRPH', 'RA', 'RVP', 'VIRT', 'WAVD', 'WKHS',
    'ZYXI']
BEST_WEIGHTED = [
    'BTTR', 'BYRN', 'CEF', 'CGAU', 'CIZN', 'CTGO', 'DYAI', 'HMY',
    'HNRG', 'IEI', 'MICS', 'MIY', 'MMU', 'MUE', 'NPV', 'NXN', 'NXP',
    'OMQS', 'PYR', 'SOBR']
SYMBOLS = sorted(list(set(SYMBOLS + LINGERERS + BEST_WEIGHTED)))

In [5]:
!ls $DL

In [6]:
#!head -5 $DL/Positions.csv

In [7]:
def check_symbols(df):
    all_present = True
    for symbol in df.index.unique():
        if symbol not in SYMBOLS:
            print(f'{symbol} not in SYMBOLS')
            all_present = False
    if all_present:
        print('All clear')

In [8]:
def convert_value(s):
    return round(float(s.replace('$', '').replace(',', '')))

In [9]:
binder = pd.DataFrame(index=SYMBOLS)
binder.head()

""
AAPL
ABMD
ABNB
ADBE
ADYEY


### E*Trade

In [10]:
TODAY = datetime.now().date()
TODAY
DONGMEI = False

In [11]:
df = pd.read_csv(
    f'{DL}/Positions.csv', index_col=0, skiprows=1
)[['Market Value']].rename(columns={'Market Value': 'ET'})
df.index = map(lambda x: x.split()[0], df.index)
df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
df.head()

,ET
AAPL,1400.9
ABMD,1542.54
ABNB,217.08
ADBE,4620.32
ADYEY,2509.78


In [13]:
if DONGMEI:
    et_df = (
        pd.read_csv(
            f'{DL}/2022-04-29.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['Portfolio', 'Cash'], inplace=True)
else:
    et_df = (
        pd.read_csv(
            f'{DL}/Positions.csv', index_col=0, skiprows=1
        )[['Market Value']]
        .rename(columns={'Market Value': 'ET'}))
    et_df.index = map(lambda x: x.split()[0], et_df.index)
    et_df.drop(index=['FNRG', 'Portfolio', 'Cash'], inplace=True)
    et_df.ET = et_df.ET.apply(
        lambda x: round(float(x)) if x != '--' else x)
et_df.head()

,ET
AAPL,1401
ABMD,1543
ABNB,217
ADBE,4620
ADYEY,2510


In [14]:
#et_df.drop(index=['98986CONT'], inplace=True)
#et_df.head()

In [15]:
et_df.loc[et_df.ET == '--']
#et_df.loc['ODFL', 'ET'] = 261.87

,ET


In [16]:
check_symbols(et_df)

All clear


In [17]:
out = pd.concat([binder, et_df], axis=1).fillna(0)
out.ET = out.ET.astype(float).apply(lambda x: int(round(x)))
out.head()

,ET
AAPL,1401
ABMD,1543
ABNB,217
ADBE,4620
ADYEY,2510


### Fidelity

In [18]:
today = datetime.strftime(TODAY, '%b-%d-%Y')
filename = f'Portfolio_Positions_{today}.csv'
filename

'Portfolio_Positions_Oct-07-2022.csv'

In [19]:
!ls $DL

Portfolio_Positions_Oct-07-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_10_07.xls


In [20]:
#df_raw = pd.read_csv(f'{DL}/{filename.replace("-16-", "-17-")}')
df_raw = pd.read_csv(f'{DL}/{filename}')
df_raw.head()

,Account Number,Account Name,Symbol,Description,Quantity,Last Price,Last Price Change,Current Value,Today's Gain/Loss Dollar,Today's Gain/Loss Percent,Total Gain/Loss Dollar,Total Gain/Loss Percent,Percent Of Account,Cost Basis,Cost Basis Per Share,Type
0,218520300,Rollover IRA,SPAXX**,FIDELITY GOVERNMENT MONEY MARKET,1282.68,$1.00,$0.00,$1282.68,NaN,NaN,NaN,NaN,3.73%,NaN,NaN,Cash
1,218520300,Rollover IRA,ABMD,ABIOMED INC COM USD0.01,4.00,$257.09,-$10.12,$1028.36,-$40.48,-3.79%,+$9.67,+0.94%,2.99%,$1018.69,$254.67,Cash
2,218520300,Rollover IRA,ADYEY,ADYEN NV UNSPON ADS EACH REP 0.01 ORD SHS,13.00,$12.74,-$1.072,$165.62,-$13.94,-7.77%,-$49.07,-22.86%,0.48%,$214.69,$16.51,Cash
3,218520300,Rollover IRA,AMD,ADVANCED MICRO DEVICES INC,9.00,$58.44,-$9.41,$525.96,-$84.69,-13.87%,-$321.09,-37.91%,1.53%,$847.05,$94.12,Cash
4,218520300,Rollover IRA,AMZN,AMAZON.COM INC,2.00,$114.56,-$5.74,$229.12,-$11.48,-4.78%,-$47.65,-17.22%,0.67%,$276.77,$138.39,Cash


In [21]:
df = df_raw[['Account Name', 'Symbol', 'Current Value']].dropna()
df['Current Value'] = df['Current Value'].apply(convert_value)
df['Current Value'] = df['Current Value'].fillna(0).astype(int)
df = df[((df.Symbol != 'SPAXX**') & (df.Symbol != 'Pending Activity'))]
df.head()

,Account Name,Symbol,Current Value
1,Rollover IRA,ABMD,1028
2,Rollover IRA,ADYEY,166
3,Rollover IRA,AMD,526
4,Rollover IRA,AMZN,229
5,Rollover IRA,AXON,371


In [22]:
accounts = {account_name: df[df['Account Name'] == account_name]
            for account_name in df['Account Name'].unique()}
for acct in accounts.keys():
    accounts[acct].index = accounts[acct].Symbol
    accounts[acct] = (
        accounts[acct]
        .rename(columns={'Current Value': acct})
        .drop(columns=['Account Name', 'Symbol']))
    
accounts['Rollover IRA'].head()

,Rollover IRA
Symbol,
ABMD,1028
ADYEY,166
AMD,526
AMZN,229
AXON,371


In [23]:
for k, v in accounts.items():
    print(k)
    check_symbols(v)
    print()

Rollover IRA
All clear

ROTH IRA
All clear

SIMPLE IRA
All clear



In [24]:
for v in accounts.values():
    out = pd.concat([out, v], axis=1)
out.fillna(0, inplace=True)
out = out.astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA
AAPL,1401,0,5604,0
ABMD,1543,1028,3342,0
ABNB,217,0,543,0
ADBE,4620,0,0,0
ADYEY,2510,166,2586,89


### TD Ameritrade

In [25]:
#!cat $DL/Positions_damiansp_Stocks_2022_08_09.xls

In [26]:
today = str(TODAY).replace('-', '_')
path = f'{DL}/Positions_damiansp_Stocks_{today}.xls'
#tomorrow = str(TODAY + timedelta(1)).replace('-', '_')
#path = f'{DL}/Positions_damiansp_Stocks_{tomorrow}.xls'

In [27]:
# Stock (default) view
#td = pd.read_excel(, engine='xlrd')
def parse_file(path):
    inds = []
    data = []
    is_header = True
    with open(path, 'r') as f:
        for line in f:
            if not is_header:
                if line.startswith('"'):
                    try:
                        columns = line.split('\t')
                        symbol = columns[0][1:-1]
                        value = float(columns[5][1:-1])
                        inds.append(symbol)
                        data.append(value)
                    except IndexError:
                        print('Cannot parse:', line)
            if line.startswith('Symbol'):
                is_header = False
    return pd.DataFrame({'TD': data}, index=inds)

In [28]:
td = parse_file(path)
td.head()

,TD
ADBE,288.60
AMRK,112.88
APT,12.03
BYND,14.09
BYRN,171.02


In [29]:
out = pd.concat([out, td], axis=1).fillna(0).round().astype(int)
out.head()

,ET,Rollover IRA,ROTH IRA,SIMPLE IRA,TD
AAPL,1401,0,5604,0,0
ABMD,1543,1028,3342,0,0
ABNB,217,0,543,0,0
ADBE,4620,0,0,0,289
ADYEY,2510,166,2586,89,0


In [30]:
out.to_csv(f'{DL}/clean.csv')

In [31]:
!open $DL/clean.csv

In [31]:
!rm $DL/$filename $DL/Positions.csv $DL/clean.csv
!ls $DL

### MarketWatch

In [10]:
!mv "$DL/Holdings - Damian Satterthwaite-Phillips.csv" "$DL/Holdings - Damian Satterthwaite-Phillips(0).csv"
!ls $DL

Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


In [11]:
dfs = [binder]
for i, f in enumerate(sorted(os.listdir(DL))):
    print(f)
    if f.startswith('Holdings'):
        mw = pd.read_csv(f'{DL}/{f}')[['Symbol', 'Value']]
        mw.rename(columns={'Value': f'Value_{i}'}, inplace=True)
        mw.index = mw.Symbol
        mw.drop('Symbol', axis=1, inplace=True)
        mw[f'Value_{i}'] = mw[f'Value_{i}'].apply(convert_value)
        dfs.append(mw)
mw = pd.concat(dfs, axis=1).fillna(0).astype(int)
mw.head()

.DS_Store
Holdings - Damian Satterthwaite-Phillips(0).csv
Holdings - Damian Satterthwaite-Phillips(1).csv
Holdings - Damian Satterthwaite-Phillips(2).csv
Portfolio_Positions_Sep-02-2022.csv
Positions.csv
Positions_damiansp_Stocks_2022_09_02.xls


,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [12]:
check_symbols(mw)

All clear


In [13]:
#out = pd.concat([binder, mw], axis=1).fillna(0).astype(int)
mw.head()

,Value_1,Value_2,Value_3
AAPL,779,2337,1558
ABMD,1045,784,0
ABNB,682,568,1136
ADBE,736,1104,0
ADYEY,602,514,1306


In [14]:
mw.to_csv(f'{DL}/mw.csv')

In [47]:
!open $DL/mw.csv

The file /Users/damiansp/Downloads/mw.csv does not exist.


In [ ]:
!rm $DL/Holdings*.csv $DL/mw.csv
!ls $DL

In [ ]:
!rm $DL/*